# Toy problem for TLS solution
We want to solve here the identification problem with TLS in a very simple case (a network with 2 nodes, without any structural contraint.

In [ ]:
import pandas as pd
import numpy as np
import cvxpy as cp
from tqdm.autonotebook import tqdm

In [ ]:
import sys
sys.path.insert(1, '..')

from src.identification.error_metrics import fro_error, error_metrics

In [ ]:
np.set_printoptions(precision=2)

In [ ]:
nodes = 4
samples = 50
v_noise_sd = 0.02
i_noise_sd = 0.02
lambda_value = 0.1
ABS_TOL = 1e-6
REL_TOL = 1e-6
MAX_ITERATIONS = 50

In [ ]:
np.random.seed(11)
y_bus = np.array([
    [1+1j, 0, 0, 0],
    [0, 2+3j, 0, 0],
    [2+1j, 0, 1+1j, 0],
    [0, 0, 0, 2+1j],
])
real_voltages = np.random.normal(1, 0.1, (samples, nodes)) + 1j*np.random.normal(1, 0.1, (samples, nodes))
real_currents = real_voltages @ y_bus

voltages = real_voltages.copy() + np.random.normal(0, v_noise_sd, (samples, nodes)) + 1j*np.random.normal(0, v_noise_sd, (samples, nodes))
currents = real_currents.copy() + np.random.normal(0, i_noise_sd, (samples, nodes)) + 1j*np.random.normal(0, i_noise_sd, (samples, nodes))

voltage_error = voltages - real_voltages
current_error = currents - real_currents

# Standard TLS

In [ ]:
def solve_tls_with_svd(A, B):
    n = A.shape[1]
    u, s, vh = np.linalg.svd(np.block([B, A]))
    v = vh.conj().T
    v_xy = v[:n, n:]
    v_yy = v[n:, n:]
    y_tls = - v_xy @ np.linalg.inv(v_yy)
    return y_tls

In [ ]:
y_tls = solve_tls_with_svd(currents, voltages)
y_tls

# L1-regularized TLS

In [ ]:
def vectorize_matrix(m):
    return m.flatten('F')

def unvectorize_matrix(v, shape):
    return np.reshape(v, shape, 'F')

def vectorize_and_make_real(m):
    m_vect = vectorize_matrix(m)
    return np.hstack([np.real(m_vect), np.imag(m_vect)])

def unvectorize_and_make_complex(v, shape):
    real_elements = shape[0] * shape[1]
    v_real = v[:real_elements]
    v_imag = v[real_elements:]
    v_complex = v_real + 1j * v_imag
    m = unvectorize_matrix(v_complex, shape)
    return m

def build_real_matrix(m):
    return np.block([
        [np.real(m), -np.imag(m)],
        [np.imag(m), np.real(m)]
    ])

In [ ]:
A = build_real_matrix(np.kron(np.eye(nodes), voltages))
dA = build_real_matrix(np.kron(np.eye(nodes), np.zeros(voltages.shape)))
a = vectorize_and_make_real(voltages)
b = vectorize_and_make_real(currents)
sigma_b = np.eye(b.size)
sigma_a = np.eye(a.size)

In [ ]:
y = cp.Variable(y_bus.size * 2)
da = cp.Variable(a.size)

In [ ]:
def lasso_target(b, A, dA, sigma_b, lambda_value, y):
    error = b - (A - dA) @ y
    loss = cp.matrix_frac(error, sigma_b) + lambda_value * cp.norm1(y) 
    return loss

def qp_target(b, a, da, sigma_b, sigma_a, undeline_y):
    error = b - undeline_y @ (a - da)
    loss = cp.matrix_frac(error, sigma_b) + cp.matrix_frac(da, sigma_a) 
    return loss

def full_target(b, a, da, sigma_b, sigma_a, lambda_value, undeline_y, y):
    return qp_target(b, a, da, sigma_b, sigma_a, underline_y) + lambda_value * cp.norm1(y)

def is_stationary_point(f_current, f_previous, abs_tol=ABS_TOL, rel_tol=REL_TOL):
    return np.abs(f_current - f_previous) < abs_tol or np.abs(f_current - f_previous) / np.abs(f_previous) < rel_tol

In [ ]:
y_lasso_res = []
e_qp_res = []
y_errors = []
e_errors = []
targets = []
for it in tqdm(range(MAX_ITERATIONS)):
    lasso_prob = cp.Problem(cp.Minimize(lasso_target(b, A, dA, sigma_b, lambda_value, y)))
    lasso_prob.solve()
    
    y_lasso = unvectorize_and_make_complex(y.value, y_bus.shape)
    underline_y = build_real_matrix(np.kron(y_lasso.T, np.eye(samples)))
    
    qp_prob = cp.Problem(cp.Minimize(qp_target(b, a, da, sigma_b, sigma_a, underline_y)))
    qp_prob.solve()
    
    e_qp = unvectorize_and_make_complex(da.value, voltages.shape)
    dA = build_real_matrix(np.kron(np.eye(nodes), e_qp))
    
    y_lasso_res.append(y_lasso)
    e_qp_res.append(e_qp)
    
    y_errors.append(fro_error(y_lasso, y_bus))
    e_errors.append(fro_error(e_qp, voltage_error))
    
    targets.append(full_target(b, a, da.value, sigma_b, sigma_a, lambda_value, underline_y, y.value).value)
    
    target_current = targets[it]
    target_previous = targets[it-1] if it > 0 else np.inf
    if is_stationary_point(target_current, target_previous):
        break

In [ ]:
pd.Series(y_errors).plot(title='Fro error on Y')

In [ ]:
pd.Series(e_errors).plot(title='Fro error on E')

In [ ]:
pd.Series(targets).plot(title='Target function')

In [ ]:
error_metrics(y_tls, y_bus)

In [ ]:
error_metrics(y_lasso, y_bus)

In [ ]:
y_tls

In [ ]:
y_lasso